# llama2 lora 16bit 实战

## Step1 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'

## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

## Step3 数据集预处理

In [4]:
tokenizer = AutoTokenizer.from_pretrained("D:/pretrained_model/LLM-Research/Llama-3___2-1B")
tokenizer
tokenizer.padding_side = "right"  # 一定要设置padding_side为right，否则batch大于1时可能不收敛

In [5]:
tokenizer.eos_token_id

128001

In [6]:
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.special_tokens_map

In [7]:
# 有bug
def process_func_bak(example):
    MAX_LENGTH = 1024
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

def process_func(example):
    MAX_LENGTH = 1024
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"])
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [8]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [9]:
print(tokenized_ds[0]["input_ids"])

[128000, 35075, 25, 111505, 69978, 113614, 9554, 126524, 46239, 3490, 72803, 25, 220, 128000, 88852, 21043, 118551, 113614, 9554, 126524, 46239, 49543, 16, 13, 111505, 69978, 111006, 108726, 1811, 74257, 36827, 102210, 108562, 40265, 9554, 111006, 114253, 116051, 107471, 65782, 5486, 110774, 65782, 58291, 83994, 126503, 3922, 27327, 113096, 42399, 64209, 104473, 36651, 113614, 3922, 50285, 103229, 120044, 107079, 120772, 91495, 19361, 103129, 35304, 111689, 83747, 33014, 30358, 3490, 17, 13, 111020, 229, 120383, 120522, 102456, 1811, 74257, 36827, 102456, 11883, 17039, 118882, 9554, 107139, 105, 108171, 5486, 53610, 28873, 5486, 37087, 104858, 53953, 34208, 121496, 57942, 103, 96412, 33857, 103167, 9554, 111678, 101828, 103706, 102456, 53953, 3922, 111098, 103048, 45736, 117587, 122603, 121496, 57942, 103, 34208, 117041, 119008, 105610, 118551, 113614, 9554, 120522, 102456, 105369, 33565, 107, 3490, 18, 13, 124022, 94, 120379, 105843, 102780, 1811, 113136, 120379, 33764, 17792, 33014, 

In [10]:
print(tokenizer.decode(tokenized_ds[2]["input_ids"]))

<|begin_of_text|>Human: 朱利叶斯·凯撒是如何死亡的？

Assistant: <|begin_of_text|>朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 

根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。<|end_of_text|>


In [11]:
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"]))))

<|begin_of_text|>4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。<|end_of_text|>


## Step4 创建模型

In [12]:
import torch 
model = AutoModelForCausalLM.from_pretrained("D:/pretrained_model/LLM-Research/Llama-3___2-1B", low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map='sequential')

In [13]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

model.embed_tokens.weight torch.float16
model.layers.0.self_attn.q_proj.weight torch.float16
model.layers.0.self_attn.k_proj.weight torch.float16
model.layers.0.self_attn.v_proj.weight torch.float16
model.layers.0.self_attn.o_proj.weight torch.float16
model.layers.0.mlp.gate_proj.weight torch.float16
model.layers.0.mlp.up_proj.weight torch.float16
model.layers.0.mlp.down_proj.weight torch.float16
model.layers.0.input_layernorm.weight torch.float16
model.layers.0.post_attention_layernorm.weight torch.float16
model.layers.1.self_attn.q_proj.weight torch.float16
model.layers.1.self_attn.k_proj.weight torch.float16
model.layers.1.self_attn.v_proj.weight torch.float16
model.layers.1.self_attn.o_proj.weight torch.float16
model.layers.1.mlp.gate_proj.weight torch.float16
model.layers.1.mlp.up_proj.weight torch.float16
model.layers.1.mlp.down_proj.weight torch.float16
model.layers.1.input_layernorm.weight torch.float16
model.layers.1.post_attention_layernorm.weight torch.float16
model.layers.2

## Lora

### PEFT Step1 配置文件

In [14]:
from peft import LoraConfig, TaskType, get_peft_model


# 可以使用qkv来指定，也可以用正则表达式进行指定
# ['q', 'v'] or '.*decoder.*(SelfAttention|EncDecAttention).*(q|v)$'."
# config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=".*\.1.*query_key_value", modules_to_save=["word_embeddings"])

config = LoraConfig(task_type=TaskType.CAUSAL_LM)
# config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=["query_key_value", "dense_4h_to_h"])
# config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=".*\.1.*query_key_value", modules_to_save=["word_embeddings"])

print(config)

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)


### PEFT Step2 创建模型

In [15]:
model = get_peft_model(model, config)

In [16]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

base_model.model.model.embed_tokens.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.float16
base_model.model.model.layers.0.self_attn.k_proj.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.float16
base_model.model.model.layers.0.self_attn.o_proj.weight torch.float16
base_model.model.model.layers.0.mlp.gate_proj.weight torch.float16
base_model.model.model.layers.0.mlp.up_proj.weight torch.float16
base_model.model.model.layers.0.mlp.down_proj.weight torch.float16
base_model.model.model.layers.0.input_layernorm.weight torch.float16
base_model.model.model.layers.0.p

In [17]:
print(config)

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='D:/pretrained_model/LLM-Research/Llama-3___2-1B', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'v_proj'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)


In [18]:
# gradient_checkpointing=True的情况下需要增加
model.enable_input_require_grads()

In [19]:
model.print_trainable_parameters()

trainable params: 851,968 || all params: 1,236,666,368 || trainable%: 0.0689


## Step5 配置训练参数

In [20]:
args = TrainingArguments(
    output_dir="./chatbot_lora_kbit",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    gradient_checkpointing=True,
    adam_epsilon=1e-5
)

## Step6 创建训练器

In [21]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

C:\Users\49207\AppData\Local\Temp\ipykernel_10824\1988846643.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Step7 模型训练

In [22]:
trainer.train()

  0%|          | 0/1678 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


{'loss': 2.2542, 'grad_norm': 0.89013671875, 'learning_rate': 4.9702026221692494e-05, 'epoch': 0.01}
{'loss': 2.1059, 'grad_norm': 1.009765625, 'learning_rate': 4.9404052443384986e-05, 'epoch': 0.01}
{'loss': 2.0694, 'grad_norm': 0.970703125, 'learning_rate': 4.910607866507748e-05, 'epoch': 0.02}
{'loss': 1.9958, 'grad_norm': 0.7880859375, 'learning_rate': 4.880810488676997e-05, 'epoch': 0.02}
{'loss': 1.9855, 'grad_norm': 0.74072265625, 'learning_rate': 4.851013110846246e-05, 'epoch': 0.03}
{'loss': 2.0075, 'grad_norm': 0.78173828125, 'learning_rate': 4.821215733015495e-05, 'epoch': 0.04}
{'loss': 2.0328, 'grad_norm': 0.81884765625, 'learning_rate': 4.791418355184744e-05, 'epoch': 0.04}
{'loss': 2.0383, 'grad_norm': 0.85888671875, 'learning_rate': 4.7616209773539935e-05, 'epoch': 0.05}
{'loss': 2.0133, 'grad_norm': 0.888671875, 'learning_rate': 4.7318235995232426e-05, 'epoch': 0.05}
{'loss': 2.0092, 'grad_norm': 0.9296875, 'learning_rate': 4.702026221692492e-05, 'epoch': 0.06}
{'loss'

KeyboardInterrupt: 

## Step8 模型推理

In [24]:
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("熬夜后第二天有哪些方法可以修复身体？中文回复我", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
print(tokenizer.decode(model.generate(**ipt, max_length=256, do_sample=True)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Human: 熬夜后第二天有哪些方法可以修复身体？中文回复我

Assistant: 在熬夜后第二天一般需要进行修复身体的方法。这些方法可以包括饮食、休息、运动、锻炼等。这些方法可以帮助身体恢复健康，减轻疲劳，并促进肌肉的发育。建议在熬夜后第二天进行这些方法，帮助身体恢复健康。


In [ ]:
import torch
torch.tensor(1e-8).half()